In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os import environ
from pathlib import Path
from typing import List

import torch
import pandas as pd
import numpy as np

from hydra import initialize, compose
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

import matplotlib.pyplot as plt

from bliss.simulator.simulated_dataset import CachedSimulatedDataset, FileDatum
from bliss.surveys.des import DarkEnergySurvey, DESDownloader



from pathlib import Path
from hydra import initialize, compose
from bliss.main import predict

In [3]:
environ["CUDA_VISIBLE_DEVICES"] = "7"

In [8]:
#environ["BLISS_HOME"] = str(Path().resolve().parents[1])
with initialize(config_path="../", version_base=None):
    cfg = compose("config", {
        "encoder.tiles_to_crop=0",
        })
    print(cfg)
#bliss_cats = predict(cfg.predict)
#bliss_cat, = bliss_cats.values()

{'paths': {'root': '${oc.env:BLISS_HOME}', 'sdss': '/data/scratch/sdss', 'decals': '/data/scratch/decals', 'des': '/data/scratch/des', 'dc2': '/data/scratch/dc2local', 'output': '${paths.root}/output'}, 'prior': {'_target_': 'case_studies.galaxy_clustering.prior.GalaxyClusterPrior', 'survey_bands': ['u', 'g', 'r', 'i', 'z'], 'reference_band': 2, 'star_color_model_path': '${simulator.survey.dir_path}/color_models/star_gmm_nmgy.pkl', 'gal_color_model_path': '${simulator.survey.dir_path}/color_models/gal_gmm_nmgy.pkl', 'n_tiles_h': 56, 'n_tiles_w': 56, 'tile_slen': 16, 'batch_size': 1, 'max_sources': 6, 'mean_sources': 0.48, 'min_sources': 0, 'prob_galaxy': 0.2, 'star_flux_exponent': 0.9859821185389767, 'star_flux_truncation': 5685.588160703261, 'star_flux_loc': -1.162430157551662, 'star_flux_scale': 1.4137911256506595, 'galaxy_flux_truncation': 1013, 'galaxy_flux_exponent': 0.47, 'galaxy_flux_scale': 0.6301037, 'galaxy_flux_loc': 0.0, 'galaxy_a_concentration': 0.39330758068481686, 'galax

In [9]:
cfg.predict

{'dataset': '${surveys.des}', 'trainer': {'_target_': 'pytorch_lightning.Trainer', 'accelerator': 'gpu', 'precision': '${train.trainer.precision}'}, 'encoder': '${encoder}', 'weight_save_path': '${paths.root}/output/version_215/checkpoints/best_encoder.ckpt', 'device': 'cuda:0'}

In [10]:
encoder = instantiate(cfg.predict.encoder)
enc_state_dict = torch.load(cfg.predict.weight_save_path)
enc_state_dict = enc_state_dict["state_dict"]
encoder.load_state_dict(enc_state_dict)
dataset = instantiate(cfg.cached_simulator)
trainer = instantiate(cfg.predict.trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
cfg.predict.encoder

{'_target_': 'case_studies.galaxy_clustering.encoder.encoder.GalaxyClusterEncoder', 'survey_bands': ['g', 'r', 'i', 'z'], 'tile_slen': '${simulator.prior.tile_slen}', 'tiles_to_crop': 0, 'min_flux_for_loss': 0, 'min_flux_for_metrics': 0, 'optimizer_params': {'lr': 0.001}, 'scheduler_params': {'milestones': [32], 'gamma': 0.1}, 'image_normalizer': {'_target_': 'bliss.encoder.image_normalizer.ImageNormalizer', 'bands': [0, 1, 2, 3], 'include_original': False, 'include_background': True, 'concat_psf_params': False, 'num_psf_params': 6, 'log_transform_stdevs': None, 'use_clahe': False, 'clahe_min_stdev': None}, 'var_dist': {'_target_': 'bliss.encoder.variational_dist.VariationalDist', 'tile_slen': '${encoder.tile_slen}', 'factors': '${variational_factors}'}, 'matcher': {'_target_': 'bliss.encoder.metrics.CatalogMatcher', 'dist_slack': 1.0, 'mag_slack': None, 'mag_band': 2}, 'metrics': {'_target_': 'torchmetrics.MetricCollection', '_convert_': 'partial', 'metrics': {'detection_performance':

In [12]:
enc_output = trainer.predict(encoder, datamodule=dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [7]


Predicting: 0it [00:00, ?it/s]

In [32]:
dataloader_iter = iter(dataset.predict_dataloader())

In [28]:
enc_output[i]

{'mode_cat': TileCatalog(16 x 20 x 20; n_sources, locs, source_type, star_fluxes, galaxy_fluxes, membership),
 'sample_cat': TileCatalog(16 x 20 x 20; n_sources, locs, source_type, star_fluxes, galaxy_fluxes, membership)}

In [33]:
for i in range(len(dataloader_iter)):
    truth = next(dataloader_iter)
    outputs = enc_output[i]
    accuracy = (outputs["mode_cat"]["membership"] == truth["tile_catalog"]["membership"]).sum()
    print(accuracy/())

tensor(0.7617)
tensor(0.7545)
tensor(0.7831)
tensor(0.6606)
tensor(0.7428)
tensor(0.7314)
tensor(0.7914)
tensor(0.7291)
tensor(0.8064)
tensor(0.7417)
tensor(0.7417)
tensor(0.7102)
tensor(0.7886)
tensor(0.8294)
tensor(0.8667)
tensor(0.6769)
tensor(0.7378)
tensor(0.7269)
tensor(0.8436)
tensor(0.7305)
tensor(0.7563)
tensor(0.8248)
tensor(0.8311)
tensor(0.7555)
tensor(0.7536)
tensor(0.7544)
tensor(0.7420)
tensor(0.8606)
tensor(0.7786)
tensor(0.7494)
tensor(0.7533)
tensor(0.8033)
tensor(0.7789)
tensor(0.8572)
tensor(0.7725)
tensor(0.7766)
tensor(0.7423)
tensor(0.8138)
tensor(0.8105)
tensor(0.7569)
tensor(0.7467)
tensor(0.7247)
tensor(0.7228)
tensor(0.8033)
tensor(0.7650)
tensor(0.8050)
tensor(0.7533)
tensor(0.7909)
tensor(0.7441)
tensor(0.7303)
tensor(0.7292)
tensor(0.7550)
tensor(0.7958)
tensor(0.7989)
tensor(0.7527)
tensor(0.7569)
tensor(0.7398)
tensor(0.7242)
tensor(0.7641)
tensor(0.7031)
tensor(0.7127)
tensor(0.7295)
tensor(0.3489)


In [19]:
(outputs["mode_cat"]["membership"] == truth["tile_catalog"]["membership"]).sum()

tensor(2233)

In [22]:
truth["tile_catalog"]["membership"].shape

torch.Size([8, 20, 20, 1, 1])

In [83]:
cma = ClusterMembershipAccuracy()
dataloader_iter = iter(dataset.predict_dataloader())
true_cat = next(dataloader_iter)
est_cat = enc_output[0]

In [87]:
from bliss.catalog import TileCatalog
target_cat = TileCatalog(20, true_cat["tile_catalog"])

In [82]:
est_cat["mode_cat"].batch_size

1

In [ ]:
cma.update()

In [47]:
first_item["tile_catalog"]["membership"].shape

torch.Size([3, 20, 20, 1, 1])

In [30]:
enc_output[0]["mode_cat"]["membership"]

torch.Size([3, 20, 20, 1, 1])

In [34]:
(enc_output[0]["mode_cat"]["n_sources"] > 0).view(12, -1).sum(dim=1)

tensor([314,   0, 276, 274,   0, 204,   0, 232,   0, 208, 269, 267])